In [1]:
from __future__ import division

import numpy as np
import os
from glob import glob

from PIL import Image
from copy import deepcopy

from sklearn import linear_model, datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

%matplotlib inline
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_context('poster')
colors = sns.color_palette("cubehelix", 5)

import pandas as pd

import scipy.stats as stats
from scipy.stats import norm
import sklearn

### define project paths

In [2]:
## add helpers to python path
import sys
if '/home/jefan/neurosketch/python' not in sys.path:
    sys.path.append('/home/jefan/neurosketch/python')

## root paths    
data_dir = '/home/jefan/neurosketch/data/'
proj_dir = '/home/jefan/neurosketch/'
results_dir = '/home/jefan/neurosketch/csv/'

## module definitions
import analysis_helpers as h
reload(h)

h.data_dir = data_dir
h.path_to_recog = data_dir + 'features/recog'
h.path_to_draw = data_dir + 'features/drawing'
h.roi_list_draw = np.array(['V1Draw', 'V2Draw', 'LOCDraw', 'InsulaDraw', 'postCentralDraw',
                            'preCentralDraw', 'ParietalDraw', 'FrontalDraw', 'smgDraw'])
h.roi_list_recog = np.array(['V1', 'V2', 'LOC', 'fusiform','parahippo','IT','ento','PRC','hipp','mOFC'])
roi_list_recog = h.roi_list_recog

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### relate individual differences in overall target selectivity (log odds of target vs. foil) to prepost differentiation

In [4]:
version = '4way'
tag = 'log'
d = pd.read_csv(os.path.join(proj_dir,'csv/difference_{}probs_{}.csv'.format(tag,version)))
prepost = pd.read_csv(os.path.join(proj_dir,'csv/neural_changes_by_surfroi_and_subject.csv'))

In [5]:
## make dataframe to relate drawing contrast to recognition differentiation
roi_list = ['V1', 'V2', 'LOC', 'IT', 'fusiform', 'parahippo', 'PRC', 'ento','hipp', 'mOFC']

for this_roi in roi_list:
    draw = d[d['roi']==this_roi]['target-foil'].values
    recog = prepost['UnanchoredTrainedDiff_{}'.format(this_roi)].values-prepost['UnanchoredControlDiff_{}'.format(this_roi)].values

    z = pd.DataFrame([draw,recog])
    z = z.transpose()
    z.columns=['draw','recog']

    ## plot 
    fig = plt.figure(figsize=(6,6))
    sns.set_context('poster')
    sns.regplot(x="draw",
                y ="recog",
                data=z)
    r,p = stats.pearsonr(draw,recog)
    plt.title('ROI: {}  r={}  p={}'.format(this_roi,np.round(r,5),np.round(p,5)))
    if np.round(p,5)<0.05:
        accent = '*'
    else:
        accent = ''
    print 'ROI: {}  r={}  p={} {}'.format(this_roi,np.round(r,5),np.round(p,5),accent)
    plt.xlabel('drawing: target vs. foil contrast') 
    plt.ylabel('recog: post-pre differentiation')
    if not os.path.exists(os.path.join(proj_dir,'plots/roi/drawrecog')):
        os.makedirs(os.path.join(proj_dir,'plots/roi/drawrecog'))
    plt.tight_layout()
    plt.savefig(os.path.join(proj_dir,'plots/roi/drawrecog/draw_recog_scatter_{}.pdf'.format(this_roi)))
    plt.close(fig)

ROI: V1  r=0.37475  p=0.03779 *
ROI: V2  r=0.46673  p=0.00812 *
ROI: LOC  r=-0.01186  p=0.94952 
ROI: IT  r=-0.10254  p=0.58306 
ROI: fusiform  r=-0.39501  p=0.02786 *
ROI: parahippo  r=0.12824  p=0.49176 
ROI: PRC  r=-0.32177  p=0.07753 
ROI: ento  r=0.08294  p=0.65735 
ROI: hipp  r=0.0656  p=0.72586 
ROI: mOFC  r=-0.3766  p=0.03677 *


### Some observations:
* ROIs where we see positive relationship are: *V1*, *V2*. That is, individuals with GREATER target selectivity during drawing in these regions also show GREATER prepost differentiation.
* ROIs where we see negative relationship are: *fusiform*, *mOFC* (maybe PRC?) That is, individuals with LESS target selectivity during drawing in these regions also show GREATER prepost differentiation.

### When does the relationship between target selectivity and differentiation emerge in these ROIs?


In [8]:
version = '4way'
tag = 'logged'
ALLDM = pd.read_csv(os.path.join(results_dir,'logistic_timeseries_drawing_neural_{}_{}.csv'.format(version,tag)))

In [10]:
ALLDM.head()

,Unnamed: 0,subj,label,run_num,TR_num,time_point,trial_num,t1_prob,t2_prob,c1_prob,c2_prob,bed_prob,bench_prob,chair_prob,table_prob,roi
0,0,110171,bed,1,10,1,0,-0.399953,-5.720615,-1.809654,-1.816093,-0.399953,-5.720615,-1.809654,-1.816093,V1
1,1,110171,bed,1,11,2,0,-0.467243,-1.571002,-1.899738,-4.146006,-0.467243,-1.571002,-1.899738,-4.146006,V1
2,2,110171,bed,1,12,3,0,-0.261171,-2.320307,-4.012254,-2.175825,-0.261171,-2.320307,-4.012254,-2.175825,V1
3,3,110171,bed,1,13,4,0,-2.709260,-3.450967,-2.398373,-0.209693,-2.709260,-3.450967,-2.398373,-0.209693,V1
4,4,110171,bed,1,14,5,0,-3.115572,-6.600511,-1.009318,-0.527939,-3.115572,-6.600511,-1.009318,-0.527939,V1


In [ ]:
def get_summary_timecourse(ALLDM, 
                        this_iv='trial_num',
                        roi_list=['V1','V2','LOC'],
                        render_cond=1,
                        version='4way',
                        proj_dir='../'):
'''
input: 
    ALLDM matrix: supersubject matrix generated by fn make_drawing_predictions
    this_iv: choice of time binning options. options are ['time_point','trial_num','run_num']
    roi_list: list of ROIs to make plots for
    render_cond: Is 1 if you want to the CONDITION-wise plots -- trained vs. foil vs control
                 Is 0 if if you want the DIFFERENCE plots -- trained - foil vs foil - control
    version: Using 4-way, 3-way, or 2-way classifier results? options are ['2way','3way','4way']

    proj_dir: root directory of project.

output: 
    saves PDF versions of plots in plots dir, which is located at top level of project directory
'''    

subs = np.unique(ALLDM.subj.values)
lookup = dict(zip(['trial_num','run_num','time_point'],['repetition','run','TR']))

ivs=['run_num','trial_num','time_point']
assert this_iv in ivs    

for this_roi in roi_list:

    print 'Now plotting results for {} ...'.format(this_roi)

    T = []
    F = []
    C = []
    Sub = []
    for sub in subs:
        inds =(ALLDM['roi']==this_roi) & (ALLDM['subj']==sub) 
        t,f,c = get_prob_timecourse(this_iv,ALLDM[inds],version=version)
        if len(T)==0:
            T = t
            F = f
            C = c
            DTF = t-f  ## these differences already in log space       
            DTC = t-c
            DFC = f-c
        else:
            T = np.hstack((T,t))
            F = np.hstack((F,f))        
            C = np.hstack((C,c)) 
            DTF = np.hstack((DTF,t-f))                
            DTC = np.hstack((DTC,t-c))
            DFC = np.hstack((DFC,f-c))
        Sub.append([sub]*len(t))   

    if render_cond==1:
        ## make longform version of dataframe to use in tsplot (by condition)            
        Trial = np.tile(np.arange(len(t)),len(subs)*3)
        Condition = np.repeat(['target','foil','control'],len(T))
        Sub = np.tile(np.array(flatten(Sub)),3)
        Prob = np.hstack((T,F,C))
        assert len(Trial)==len(Condition)
        assert len(Sub)==len(Prob)
        assert len(Condition)==len(Sub)
        x = pd.DataFrame([Prob,Trial,Condition,Sub])
        x = x.transpose()
        x.columns = ['probability',lookup[this_iv],'condition','sub']
        toop = 'condition'            
    else:
        ## make longform version of dataframe to use in tsplot (difference btw conditions)                    
        Trial = np.tile(np.arange(len(t)),len(subs)*3)
        Condition = np.repeat(['target-foil','target-control','foil-control'],len(T))
        Sub = np.tile(np.array(flatten(Sub)),3)
        Prob = np.hstack((DTF,DTC,DFC))        
        assert len(Trial)==len(Condition)
        assert len(Sub)==len(Prob)
        assert len(Condition)==len(Sub)
        x = pd.DataFrame([Prob,Trial,Condition,Sub])
        x = x.transpose()
        x.columns = ['probability',lookup[this_iv],'condition','sub']        
        toop = 'difference'